In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests as r
from bs4 import BeautifulSoup



#web scraping wikipedia page for highest selling video games
url = 'https://en.wikipedia.org/wiki/List_of_best-selling_video_games'
unparsed_wiki = r.get(url)

#parsing data into a beautifulsoup object
soup = BeautifulSoup(unparsed_wiki.text, 'html.parser')
parsed_wiki = soup.find('table',{'class':"wikitable"})

#converting parsed data into a dataframe
videogame_sales = pd.read_html(str(parsed_wiki))
vg_sales = pd.DataFrame(videogame_sales[0])

#dropping 'Ref' column. inplace = True so to not create new dataframe
vg_sales.drop(columns = ['Ref'], inplace=True)
#renaming columns to remove superfluous syntax 
vg_sales.rename(columns={"Platform(s)": "Platform", "Initial release date": "date", "Developer(s)[b]": "Developer", "Publisher(s)[b]": "Publisher"}, inplace = True)

#importing data from two .csv (video game inspired movies)
highest_gross_movies = pd.read_csv('/Users/jaimepira/Desktop/highest_gross_video_game_films.csv')
video_game_movies = pd.read_csv('/Users/jaimepira/Desktop/video_game_films.csv')

#merging the two df to create a more complete list of movies inspired by video games
vg_movies = pd.merge(video_game_movies, highest_gross_movies, how='outer')
#removing duplicates and irrelevant data column
vg_movies.drop_duplicates(subset="Title")
vg_movies.drop(columns=['Rank', 'Peak'], inplace=True)





print(vg_movies.head(40))

                                            Title Release date  \
0                               Super Mario Bros.   1993-05-28   
1                                   Double Dragon   1994-11-04   
2                                  Street Fighter   1994-12-23   
3                                   Mortal Kombat   1995-08-18   
4                     Mortal Kombat: Annihilation   1997-11-21   
5                                  Wing Commander   1999-03-12   
6                         Lara Croft: Tomb Raider   2001-06-15   
7                                   Resident Evil   2002-03-15   
8    Lara Croft: Tomb Raider – The Cradle of Life   2003-07-25   
9                               House of the Dead   2003-10-10   
10                      Resident Evil: Apocalypse   2004-09-10   
11                              Alone in the Dark   2005-01-28   
12                                           Doom   2005-10-21   
13                                     BloodRayne   2006-01-06   
14        